# Deep Dive 05: 极端倾向得分的诊断与处理

## 🎯 学习目标

完成本 notebook 后，你将能够：

1. **诊断** 极端倾向得分问题（0.99 或 0.01）
2. **理解** 极端权重对因果估计的影响
3. **掌握** 多种处理方法：Trimming、Overlap Weights、CRUMP bounds
4. **选择** 适合具体场景的最佳方案

---

## 📖 背景故事

> **场景：IPW 估计的奇怪结果**
>
> 你在评估一个用户激励策略的效果，使用 IPW 估计 ATE。结果发现：
> - 标准误异常大
> - 估计值不稳定（每次运行差别很大）
> - 置信区间宽得离谱
>
> 检查倾向得分分布后，你发现问题了：
> - 有些用户的倾向得分 e(x) = 0.99（几乎必然被处理）
> - 有些用户的倾向得分 e(x) = 0.01（几乎不可能被处理）
>
> **这些极端值导致 IPW 权重爆炸！本 notebook 将解决这个问题。**

---

## Part 1: 问题诊断

### 1.1 为什么极端倾向得分是问题？

回顾 IPW 估计量：

$$\hat{\tau}_{IPW} = \frac{1}{n} \sum_{i} \left[ \frac{T_i Y_i}{e(X_i)} - \frac{(1-T_i) Y_i}{1-e(X_i)} \right]$$

**问题**：
- 当 $e(X_i) \approx 0$，权重 $1/e(X_i) \to \infty$
- 当 $e(X_i) \approx 1$，权重 $1/(1-e(X_i)) \to \infty$

**后果**：
1. **方差爆炸**：少数样本主导估计
2. **估计不稳定**：对极端样本高度敏感
3. **置信区间失效**：实际覆盖率远低于名义水平

In [ ]:
# 环境准备
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

# 设置绘图风格
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.family'] = ['Arial Unicode MS', 'SimHei', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False

print("✅ 环境准备完成")

In [ ]:
def generate_extreme_propensity_data(n=2000, seed=42):
    """
    生成带有极端倾向得分的数据
    
    场景：高价值用户几乎必然被处理（e(x) ≈ 1）
         低价值用户几乎不被处理（e(x) ≈ 0）
    """
    np.random.seed(seed)
    
    # 协变量
    X = np.random.randn(n, 3)
    X[:, 0] = X[:, 0] * 2  # 第一个特征影响更大
    
    # 真实倾向得分（故意制造极端值）
    logit = 3 * X[:, 0] + 1.5 * X[:, 1]  # 强依赖于协变量
    true_propensity = 1 / (1 + np.exp(-logit))
    
    # 处理分配
    T = np.random.binomial(1, true_propensity)
    
    # 真实处理效应
    true_ate = 5.0
    true_cate = 5 + 2 * X[:, 0]  # 异质性效应
    
    # 结果
    Y0 = 10 + 2 * X[:, 0] + X[:, 1] + np.random.randn(n) * 2
    Y1 = Y0 + true_cate
    Y = T * Y1 + (1 - T) * Y0
    
    return {
        'X': X,
        'T': T,
        'Y': Y,
        'true_propensity': true_propensity,
        'true_ate': true_ate,
        'true_cate': true_cate,
        'Y0': Y0,
        'Y1': Y1
    }

# 生成数据
data = generate_extreme_propensity_data(n=3000)

print("数据概况：")
print(f"  样本数: {len(data['Y'])}")
print(f"  处理组: {data['T'].sum()} ({data['T'].mean():.1%})")
print(f"  真实 ATE: {data['true_ate']}")
print(f"\n倾向得分分布:")
print(f"  最小值: {data['true_propensity'].min():.4f}")
print(f"  最大值: {data['true_propensity'].max():.4f}")
print(f"  < 0.05: {(data['true_propensity'] < 0.05).sum()} ({(data['true_propensity'] < 0.05).mean():.1%})")
print(f"  > 0.95: {(data['true_propensity'] > 0.95).sum()} ({(data['true_propensity'] > 0.95).mean():.1%})")

In [ ]:
# 可视化倾向得分分布
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# 倾向得分直方图
ax1 = axes[0]
ax1.hist(data['true_propensity'], bins=50, edgecolor='black', alpha=0.7)
ax1.axvline(x=0.1, color='red', linestyle='--', label='危险区域边界 (0.1)')
ax1.axvline(x=0.9, color='red', linestyle='--')
ax1.fill_betweenx([0, 500], 0, 0.1, alpha=0.3, color='red')
ax1.fill_betweenx([0, 500], 0.9, 1, alpha=0.3, color='red')
ax1.set_xlabel('倾向得分 e(X)')
ax1.set_ylabel('频数')
ax1.set_title('倾向得分分布\n(红色区域为极端值)')
ax1.legend()

# 按处理组分布
ax2 = axes[1]
ax2.hist(data['true_propensity'][data['T']==0], bins=30, alpha=0.6, 
         label='控制组', color='#3498db')
ax2.hist(data['true_propensity'][data['T']==1], bins=30, alpha=0.6, 
         label='处理组', color='#e74c3c')
ax2.set_xlabel('倾向得分 e(X)')
ax2.set_ylabel('频数')
ax2.set_title('按处理组的倾向得分分布')
ax2.legend()

# IPW 权重分布
ax3 = axes[2]
weights_treated = 1 / data['true_propensity'][data['T']==1]
weights_control = 1 / (1 - data['true_propensity'][data['T']==0])

ax3.hist(weights_treated, bins=50, alpha=0.6, label='处理组权重 1/e(X)', color='#e74c3c')
ax3.hist(weights_control, bins=50, alpha=0.6, label='控制组权重 1/(1-e(X))', color='#3498db')
ax3.set_xlabel('IPW 权重')
ax3.set_ylabel('频数')
ax3.set_title('IPW 权重分布\n(注意极端值!)')
ax3.set_xlim(0, 50)  # 限制显示范围
ax3.legend()

plt.tight_layout()
plt.show()

print(f"\n权重统计:")
print(f"  处理组权重最大值: {weights_treated.max():.1f}")
print(f"  控制组权重最大值: {weights_control.max():.1f}")
print(f"  ⚠️ 单个样本的权重可能超过 100 倍平均权重!")

### 1.2 有效样本量 (ESS)

有效样本量衡量加权后样本的"有效"数量：

$$\text{ESS} = \frac{(\sum_i w_i)^2}{\sum_i w_i^2}$$

当权重极端不平衡时，ESS 会远小于实际样本量。

In [ ]:
def calculate_ess(weights):
    """计算有效样本量"""
    return (np.sum(weights) ** 2) / np.sum(weights ** 2)

def diagnose_propensity_scores(e, T):
    """
    倾向得分诊断报告
    """
    print("=" * 60)
    print("倾向得分诊断报告")
    print("=" * 60)
    
    # 基本统计
    print(f"\n1. 基本统计")
    print(f"   样本量: {len(e)}")
    print(f"   处理组: {T.sum()} ({T.mean():.1%})")
    print(f"   倾向得分范围: [{e.min():.4f}, {e.max():.4f}]")
    
    # 极端值检测
    print(f"\n2. 极端值检测")
    thresholds = [0.01, 0.05, 0.1]
    for th in thresholds:
        n_low = (e < th).sum()
        n_high = (e > 1-th).sum()
        print(f"   e < {th:.2f} 或 e > {1-th:.2f}: {n_low + n_high} ({(n_low + n_high)/len(e):.1%})")
    
    # 有效样本量
    print(f"\n3. 有效样本量 (ESS)")
    
    # 处理组
    weights_t = 1 / e[T==1]
    ess_t = calculate_ess(weights_t)
    print(f"   处理组: ESS = {ess_t:.1f} / {T.sum()} ({ess_t/T.sum():.1%})")
    
    # 控制组
    weights_c = 1 / (1 - e[T==0])
    ess_c = calculate_ess(weights_c)
    print(f"   控制组: ESS = {ess_c:.1f} / {(1-T).sum()} ({ess_c/(1-T).sum():.1%})")
    
    # 推荐
    print(f"\n4. 诊断结论")
    if ess_t/T.sum() < 0.5 or ess_c/(1-T).sum() < 0.5:
        print("   ⚠️ 警告: 有效样本量损失严重 (>50%)，建议处理极端权重")
    elif ess_t/T.sum() < 0.8 or ess_c/(1-T).sum() < 0.8:
        print("   ⚡ 注意: 有效样本量有一定损失 (20-50%)，可考虑处理极端权重")
    else:
        print("   ✅ 有效样本量损失较小 (<20%)，可直接使用 IPW")

# 运行诊断
diagnose_propensity_scores(data['true_propensity'], data['T'])

---

## Part 2: 处理方法

### 2.1 方法总览

| 方法 | 核心思想 | 优点 | 缺点 |
|------|---------|------|------|
| Trimming | 删除极端样本 | 简单直接 | 丢失样本，可能有偏 |
| Truncated IPW | 截断极端权重 | 减少方差 | 引入偏差 |
| Overlap Weights | 优化权重设计 | 理论最优 | 估计 ATO 非 ATE |
| Stabilized Weights | 稳定化权重 | 减少方差 | 仍可能有极端值 |
| CRUMP Bounds | 数据驱动阈值 | 有理论支撑 | 可能过于保守 |

### 2.2 方法 1: Trimming（修剪）

最简单的方法：删除倾向得分在极端区域的样本。

$$\text{保留样本: } \alpha < e(X_i) < 1 - \alpha$$

常见阈值：$\alpha = 0.1$ 或 $\alpha = 0.05$

In [ ]:
def ipw_with_trimming(Y, T, e, trim_threshold=0.1):
    """
    带 Trimming 的 IPW 估计
    
    Args:
        Y: 结果
        T: 处理
        e: 倾向得分
        trim_threshold: 修剪阈值
        
    Returns:
        ate: ATE 估计
        se: 标准误
        n_trimmed: 被修剪的样本数
    """
    # 修剪
    keep_mask = (e > trim_threshold) & (e < 1 - trim_threshold)
    n_trimmed = (~keep_mask).sum()
    
    Y_trim = Y[keep_mask]
    T_trim = T[keep_mask]
    e_trim = e[keep_mask]
    
    # IPW 估计
    weights_t = T_trim / e_trim
    weights_c = (1 - T_trim) / (1 - e_trim)
    
    # 归一化权重
    weights_t = weights_t / weights_t.sum() * len(weights_t)
    weights_c = weights_c / weights_c.sum() * len(weights_c)
    
    mu1 = np.sum(weights_t * Y_trim * T_trim) / np.sum(weights_t * T_trim)
    mu0 = np.sum(weights_c * Y_trim * (1-T_trim)) / np.sum(weights_c * (1-T_trim))
    ate = mu1 - mu0
    
    # 标准误（简化版本）
    n = len(Y_trim)
    var_t = np.var(Y_trim[T_trim==1], ddof=1) / (T_trim==1).sum()
    var_c = np.var(Y_trim[T_trim==0], ddof=1) / (T_trim==0).sum()
    se = np.sqrt(var_t + var_c)
    
    return ate, se, n_trimmed

# 测试不同阈值
thresholds = [0.01, 0.05, 0.1, 0.15, 0.2]
results_trimming = []

for th in thresholds:
    ate, se, n_trimmed = ipw_with_trimming(data['Y'], data['T'], data['true_propensity'], th)
    results_trimming.append({
        'threshold': th,
        'ate': ate,
        'se': se,
        'n_trimmed': n_trimmed,
        'pct_trimmed': n_trimmed / len(data['Y'])
    })

# 显示结果
print("\nTrimming 方法结果 (真实 ATE = 5.0):")
print("=" * 70)
print(f"{'阈值':<10} {'ATE 估计':<12} {'标准误':<12} {'修剪样本数':<15} {'修剪比例':<10}")
print("-" * 70)
for r in results_trimming:
    print(f"{r['threshold']:<10} {r['ate']:<12.3f} {r['se']:<12.3f} {r['n_trimmed']:<15} {r['pct_trimmed']:<10.1%}")

### 2.3 方法 2: Overlap Weights

Overlap Weights (Li et al., 2018) 使用最优权重设计，避免极端值。

**权重定义**：
$$w(X) = e(X)(1 - e(X))$$

**估计目标**：ATO (Average Treatment Effect on the Overlap population)
$$\tau_{ATO} = E[Y(1) - Y(0) | \text{有重叠}]$$

**优点**：
- 自动下调极端样本的权重
- 最小化估计方差
- 不需要人工选择阈值

In [ ]:
def ipw_with_overlap_weights(Y, T, e):
    """
    使用 Overlap Weights 的 IPW 估计
    
    估计 ATO: Average Treatment Effect on the Overlap population
    """
    n = len(Y)
    
    # Overlap 权重: w(X) = e(X) * (1 - e(X))
    overlap_weights = e * (1 - e)
    
    # 处理组权重: (1 - e(X)) 对于被处理的个体
    # 控制组权重: e(X) 对于未被处理的个体
    weights_t = (1 - e) * T  # 处理组
    weights_c = e * (1 - T)  # 控制组
    
    # 估计 ATO
    mu1 = np.sum(weights_t * Y) / np.sum(weights_t)
    mu0 = np.sum(weights_c * Y) / np.sum(weights_c)
    ato = mu1 - mu0
    
    # 有效样本量
    ess_t = calculate_ess(weights_t[T==1])
    ess_c = calculate_ess(weights_c[T==0])
    
    # 标准误
    residuals_t = (Y - mu1) * weights_t
    residuals_c = (Y - mu0) * weights_c
    var_t = np.sum(residuals_t**2) / (np.sum(weights_t)**2)
    var_c = np.sum(residuals_c**2) / (np.sum(weights_c)**2)
    se = np.sqrt(var_t + var_c)
    
    return {
        'ato': ato,
        'se': se,
        'ess_treated': ess_t,
        'ess_control': ess_c
    }

# 计算 Overlap Weights 结果
result_overlap = ipw_with_overlap_weights(data['Y'], data['T'], data['true_propensity'])

print("\nOverlap Weights 结果:")
print("=" * 50)
print(f"  ATO 估计: {result_overlap['ato']:.3f}")
print(f"  标准误: {result_overlap['se']:.3f}")
print(f"  95% CI: [{result_overlap['ato']-1.96*result_overlap['se']:.3f}, "
      f"{result_overlap['ato']+1.96*result_overlap['se']:.3f}]")
print(f"  处理组 ESS: {result_overlap['ess_treated']:.1f}")
print(f"  控制组 ESS: {result_overlap['ess_control']:.1f}")

### 2.4 方法 3: Stabilized Weights

Stabilized Weights 将权重乘以边际处理概率，减少方差。

$$w^{stab} = \frac{P(T)}{e(X)} \cdot T + \frac{1 - P(T)}{1 - e(X)} \cdot (1-T)$$

In [ ]:
def ipw_with_stabilized_weights(Y, T, e):
    """
    使用 Stabilized Weights 的 IPW 估计
    """
    n = len(Y)
    
    # 边际处理概率
    p_treat = T.mean()
    
    # Stabilized weights
    weights = np.where(T == 1, 
                       p_treat / e,
                       (1 - p_treat) / (1 - e))
    
    # 加权均值
    mu1 = np.sum(weights * Y * T) / np.sum(weights * T)
    mu0 = np.sum(weights * Y * (1-T)) / np.sum(weights * (1-T))
    ate = mu1 - mu0
    
    # 有效样本量
    ess_t = calculate_ess(weights[T==1])
    ess_c = calculate_ess(weights[T==0])
    
    # 标准误
    n_t = (T==1).sum()
    n_c = (T==0).sum()
    var_t = np.sum((weights * (Y - mu1))**2 * T) / (n_t * (n_t - 1))
    var_c = np.sum((weights * (Y - mu0))**2 * (1-T)) / (n_c * (n_c - 1))
    se = np.sqrt(var_t + var_c)
    
    return {
        'ate': ate,
        'se': se,
        'ess_treated': ess_t,
        'ess_control': ess_c,
        'max_weight': weights.max()
    }

# 计算 Stabilized Weights 结果
result_stabilized = ipw_with_stabilized_weights(data['Y'], data['T'], data['true_propensity'])

print("\nStabilized Weights 结果:")
print("=" * 50)
print(f"  ATE 估计: {result_stabilized['ate']:.3f}")
print(f"  标准误: {result_stabilized['se']:.3f}")
print(f"  处理组 ESS: {result_stabilized['ess_treated']:.1f}")
print(f"  控制组 ESS: {result_stabilized['ess_control']:.1f}")
print(f"  最大权重: {result_stabilized['max_weight']:.2f}")

### 2.5 方法 4: CRUMP Bounds

CRUMP et al. (2009) 提出数据驱动的阈值选择方法。

**核心思想**：选择阈值使估计方差最小化。

**近似最优阈值**：
$$\alpha^* \approx \frac{1}{2} - \sqrt{\frac{1}{4} - \frac{1}{\sqrt{n}}}$$

In [ ]:
def crump_threshold(n):
    """
    计算 CRUMP 推荐的阈值
    """
    # 近似公式
    inner = 0.25 - 1 / np.sqrt(n)
    if inner < 0:
        return 0.1  # 默认值
    return 0.5 - np.sqrt(inner)

def ipw_with_crump(Y, T, e):
    """
    使用 CRUMP bounds 的 IPW 估计
    """
    n = len(Y)
    threshold = crump_threshold(n)
    
    return ipw_with_trimming(Y, T, e, trim_threshold=threshold), threshold

# 计算 CRUMP 结果
(ate_crump, se_crump, n_trimmed_crump), threshold_crump = ipw_with_crump(
    data['Y'], data['T'], data['true_propensity']
)

print(f"\nCRUMP Bounds 结果:")
print("=" * 50)
print(f"  推荐阈值: {threshold_crump:.4f}")
print(f"  ATE 估计: {ate_crump:.3f}")
print(f"  标准误: {se_crump:.3f}")
print(f"  修剪样本: {n_trimmed_crump} ({n_trimmed_crump/len(data['Y']):.1%})")

---

## Part 3: 方法比较

In [ ]:
# 标准 IPW（无处理）
def standard_ipw(Y, T, e):
    weights_t = T / e
    weights_c = (1 - T) / (1 - e)
    
    mu1 = np.sum(weights_t * Y) / np.sum(weights_t)
    mu0 = np.sum(weights_c * Y) / np.sum(weights_c)
    
    ate = mu1 - mu0
    
    # 简化标准误
    se = np.sqrt(np.var(Y[T==1])/T.sum() + np.var(Y[T==0])/(1-T).sum())
    
    return ate, se

ate_standard, se_standard = standard_ipw(data['Y'], data['T'], data['true_propensity'])

# 汇总所有方法
methods_comparison = [
    {'方法': '标准 IPW', 'ATE/ATO': ate_standard, '标准误': se_standard, '目标': 'ATE'},
    {'方法': 'Trimming (α=0.1)', 'ATE/ATO': results_trimming[2]['ate'], 
     '标准误': results_trimming[2]['se'], '目标': 'ATE (trimmed)'},
    {'方法': 'Overlap Weights', 'ATE/ATO': result_overlap['ato'], 
     '标准误': result_overlap['se'], '目标': 'ATO'},
    {'方法': 'Stabilized Weights', 'ATE/ATO': result_stabilized['ate'], 
     '标准误': result_stabilized['se'], '目标': 'ATE'},
    {'方法': 'CRUMP Bounds', 'ATE/ATO': ate_crump, 
     '标准误': se_crump, '目标': 'ATE (optimal trim)'},
]

print("\n" + "=" * 75)
print("方法比较汇总 (真实 ATE = 5.0)")
print("=" * 75)
print(f"\n{'方法':<25} {'估计值':<12} {'标准误':<12} {'估计目标':<20}")
print("-" * 70)
for m in methods_comparison:
    print(f"{m['方法']:<25} {m['ATE/ATO']:<12.3f} {m['标准误']:<12.3f} {m['目标']:<20}")

In [ ]:
# 可视化比较
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 估计值和置信区间
ax1 = axes[0]
methods = [m['方法'] for m in methods_comparison]
estimates = [m['ATE/ATO'] for m in methods_comparison]
ses = [m['标准误'] for m in methods_comparison]

y_pos = np.arange(len(methods))
colors = ['#e74c3c', '#3498db', '#2ecc71', '#f39c12', '#9b59b6']

ax1.barh(y_pos, estimates, xerr=[1.96*s for s in ses], 
         color=colors, capsize=5, alpha=0.7)
ax1.axvline(x=5.0, color='red', linestyle='--', linewidth=2, label='真实 ATE = 5.0')
ax1.set_yticks(y_pos)
ax1.set_yticklabels(methods)
ax1.set_xlabel('估计值 ± 95% CI')
ax1.set_title('不同方法的估计值和置信区间')
ax1.legend()

# 标准误比较
ax2 = axes[1]
bars = ax2.bar(range(len(methods)), ses, color=colors, alpha=0.7)
ax2.set_xticks(range(len(methods)))
ax2.set_xticklabels(methods, rotation=45, ha='right')
ax2.set_ylabel('标准误')
ax2.set_title('不同方法的标准误')

# 添加数值标签
for bar, se in zip(bars, ses):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05, 
            f'{se:.2f}', ha='center', fontsize=9)

plt.tight_layout()
plt.show()

---

## Part 4: 模拟研究 - 哪种方法最好？

In [ ]:
def simulation_study(n_simulations=200, n_samples=1000):
    """
    模拟研究：比较不同方法的偏差和覆盖率
    """
    results = {
        'Standard IPW': {'estimates': [], 'covered': []},
        'Trimming (0.1)': {'estimates': [], 'covered': []},
        'Overlap Weights': {'estimates': [], 'covered': []},
        'Stabilized Weights': {'estimates': [], 'covered': []},
    }
    
    true_ate = 5.0
    
    for sim in range(n_simulations):
        # 生成数据
        data = generate_extreme_propensity_data(n=n_samples, seed=sim)
        Y, T, e = data['Y'], data['T'], data['true_propensity']
        
        # Standard IPW
        ate, se = standard_ipw(Y, T, e)
        results['Standard IPW']['estimates'].append(ate)
        results['Standard IPW']['covered'].append(
            (true_ate >= ate - 1.96*se) and (true_ate <= ate + 1.96*se)
        )
        
        # Trimming
        ate, se, _ = ipw_with_trimming(Y, T, e, trim_threshold=0.1)
        results['Trimming (0.1)']['estimates'].append(ate)
        results['Trimming (0.1)']['covered'].append(
            (true_ate >= ate - 1.96*se) and (true_ate <= ate + 1.96*se)
        )
        
        # Overlap Weights (注意：估计 ATO，不是 ATE)
        result = ipw_with_overlap_weights(Y, T, e)
        results['Overlap Weights']['estimates'].append(result['ato'])
        results['Overlap Weights']['covered'].append(
            (true_ate >= result['ato'] - 1.96*result['se']) and 
            (true_ate <= result['ato'] + 1.96*result['se'])
        )
        
        # Stabilized Weights
        result = ipw_with_stabilized_weights(Y, T, e)
        results['Stabilized Weights']['estimates'].append(result['ate'])
        results['Stabilized Weights']['covered'].append(
            (true_ate >= result['ate'] - 1.96*result['se']) and 
            (true_ate <= result['ate'] + 1.96*result['se'])
        )
    
    # 汇总
    summary = []
    for method, res in results.items():
        estimates = np.array(res['estimates'])
        summary.append({
            'method': method,
            'mean': np.mean(estimates),
            'bias': np.mean(estimates) - true_ate,
            'std': np.std(estimates),
            'rmse': np.sqrt(np.mean((estimates - true_ate)**2)),
            'coverage': np.mean(res['covered'])
        })
    
    return summary, results

# 运行模拟
print("运行模拟研究 (这可能需要一些时间)...")
summary, sim_results = simulation_study(n_simulations=200, n_samples=1000)

print("\n" + "=" * 80)
print(f"模拟研究结果 (200 次模拟, 真实 ATE = 5.0)")
print("=" * 80)
print(f"\n{'方法':<25} {'均值':<10} {'偏差':<10} {'标准差':<10} {'RMSE':<10} {'覆盖率':<10}")
print("-" * 75)
for s in summary:
    print(f"{s['method']:<25} {s['mean']:<10.3f} {s['bias']:<10.3f} {s['std']:<10.3f} "
          f"{s['rmse']:<10.3f} {s['coverage']:<10.1%}")

In [ ]:
# 可视化模拟结果
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

methods = [s['method'] for s in summary]
colors = ['#e74c3c', '#3498db', '#2ecc71', '#f39c12']

# 估计分布
ax1 = axes[0]
for i, (method, color) in enumerate(zip(methods, colors)):
    estimates = sim_results[method]['estimates']
    ax1.hist(estimates, bins=20, alpha=0.5, label=method, color=color)
ax1.axvline(x=5.0, color='black', linestyle='--', linewidth=2, label='真实 ATE')
ax1.set_xlabel('ATE 估计')
ax1.set_ylabel('频数')
ax1.set_title('估计值分布')
ax1.legend(fontsize=8)

# RMSE 比较
ax2 = axes[1]
rmses = [s['rmse'] for s in summary]
bars = ax2.bar(range(len(methods)), rmses, color=colors)
ax2.set_xticks(range(len(methods)))
ax2.set_xticklabels(methods, rotation=45, ha='right')
ax2.set_ylabel('RMSE')
ax2.set_title('RMSE 比较')

# 覆盖率比较
ax3 = axes[2]
coverages = [s['coverage'] for s in summary]
bars = ax3.bar(range(len(methods)), coverages, color=colors)
ax3.axhline(y=0.95, color='red', linestyle='--', label='名义水平 95%')
ax3.set_xticks(range(len(methods)))
ax3.set_xticklabels(methods, rotation=45, ha='right')
ax3.set_ylabel('95% CI 覆盖率')
ax3.set_title('覆盖率比较')
ax3.set_ylim(0, 1)
ax3.legend()

plt.tight_layout()
plt.show()

---

## Part 5: 最佳实践流程

### 🧪 TODO 练习: 实现完整的诊断和处理流程

In [ ]:
<cell_type>markdown</cell_type>### 🧪 TODO 练习: 实现完整的诊断和处理流程

请实现一个自动化的倾向得分处理流程，能够：
1. 估计倾向得分
2. 诊断极端值问题
3. 自动选择最佳处理方法
4. 输出估计结果

def propensity_score_pipeline(Y, T, X, method='auto'):
    """
    完整的倾向得分处理流程

    Args:
        Y: 结果
        T: 处理
        X: 协变量
        method: 处理方法 ('auto', 'trimming', 'overlap', 'stabilized')

    Returns:
        result: 估计结果和诊断信息
    """
    # Step 1 - 估计倾向得分
    # 使用 LogisticRegression 拟合
    ps_model = LogisticRegression(max_iter=1000)
    ps_model.fit(X, T)
    e = ps_model.predict_proba(X)[:, 1]

    # Step 2 - 诊断
    print("\n" + "=" * 60)
    print("Step 1: 倾向得分估计完成")
    print("=" * 60)
    diagnose_propensity_scores(e, T)

    # Step 3 - 自动选择方法
    # 根据 ESS 损失程度选择方法
    # - ESS 损失 > 70%: overlap
    # - ESS 损失 40-70%: trimming
    # - ESS 损失 < 40%: stabilized

    if method == 'auto':
        # 计算 ESS 损失
        weights_t = 1 / e[T==1]
        weights_c = 1 / (1 - e[T==0])
        ess_t = calculate_ess(weights_t)
        ess_c = calculate_ess(weights_c)
        
        ess_loss_t = 1 - ess_t / (T==1).sum()
        ess_loss_c = 1 - ess_c / (T==0).sum()
        ess_loss = max(ess_loss_t, ess_loss_c)
        
        if ess_loss > 0.7:
            method = 'overlap'
        elif ess_loss > 0.4:
            method = 'trimming'
        else:
            method = 'stabilized'

    print(f"\n选择的方法: {method}")

    # Step 4 - 执行估计
    # 根据 method 调用相应函数
    print("\n" + "=" * 60)
    print(f"Step 2: 使用 {method} 方法估计")
    print("=" * 60)
    
    if method == 'trimming':
        ate, se, n_trimmed = ipw_with_trimming(Y, T, e, trim_threshold=0.1)
        result = {
            'method': 'Trimming (α=0.1)',
            'estimate': ate,
            'se': se,
            'ci_lower': ate - 1.96 * se,
            'ci_upper': ate + 1.96 * se,
            'n_trimmed': n_trimmed
        }
        print(f"  ATE 估计: {ate:.3f}")
        print(f"  标准误: {se:.3f}")
        print(f"  95% CI: [{result['ci_lower']:.3f}, {result['ci_upper']:.3f}]")
        print(f"  修剪样本: {n_trimmed} ({n_trimmed/len(Y):.1%})")
        
    elif method == 'overlap':
        result_dict = ipw_with_overlap_weights(Y, T, e)
        result = {
            'method': 'Overlap Weights',
            'estimate': result_dict['ato'],
            'se': result_dict['se'],
            'ci_lower': result_dict['ato'] - 1.96 * result_dict['se'],
            'ci_upper': result_dict['ato'] + 1.96 * result_dict['se'],
            'ess_treated': result_dict['ess_treated'],
            'ess_control': result_dict['ess_control']
        }
        print(f"  ATO 估计: {result['estimate']:.3f}")
        print(f"  标准误: {result['se']:.3f}")
        print(f"  95% CI: [{result['ci_lower']:.3f}, {result['ci_upper']:.3f}]")
        print(f"  处理组 ESS: {result['ess_treated']:.1f}")
        print(f"  控制组 ESS: {result['ess_control']:.1f}")
        
    elif method == 'stabilized':
        result_dict = ipw_with_stabilized_weights(Y, T, e)
        result = {
            'method': 'Stabilized Weights',
            'estimate': result_dict['ate'],
            'se': result_dict['se'],
            'ci_lower': result_dict['ate'] - 1.96 * result_dict['se'],
            'ci_upper': result_dict['ate'] + 1.96 * result_dict['se'],
            'ess_treated': result_dict['ess_treated'],
            'ess_control': result_dict['ess_control']
        }
        print(f"  ATE 估计: {result['estimate']:.3f}")
        print(f"  标准误: {result['se']:.3f}")
        print(f"  95% CI: [{result['ci_lower']:.3f}, {result['ci_upper']:.3f}]")
        print(f"  处理组 ESS: {result['ess_treated']:.1f}")
        print(f"  控制组 ESS: {result['ess_control']:.1f}")
    else:
        result = None

    return result, e

print("✅ propensity_score_pipeline 函数定义完成（已完成 TODO 部分）")